# Laboratorio 2

## Desarrollo de una herramienta analítica usando paquetes especializados para análisis de datos en Python

Para el desarrollo de esta actividad puedes utilizar cualquier librería externa. Te recomendamos leer por completo el enunciado del laboratorio antes de comenzar, de forma que tengas claro el propósito global de la actividad y puedas desarrollar tu solución apuntando a él desde el inicio.

Al desarrollar este laboratorio pondrás a prueba tus habilidades para:

1. Identificar y abordar preguntas de negocio y de *analytics*.
2. Leer datos desde archivos y almacenarlos utilizando métodos de librerías especializadas.
3. Explorar, modificar, limpiar y unir objetos tablas de datos.
4. Implementar análisis combinando métricas descriptivas, visualización, filtrado y agrupación.
5. Implementar análisis basado en modelos estadísticos o de *machine learning*.

##  Contexto: desigualdad y factores de éxito en pruebas Saber 11 en Colombia

El ICFES es el Instituto Colombiano para el Fomento de la Educación Superior y está adscrito al Ministerio de Educación a nivel nacional. Como parte de sus funciones, el ICFES administra las pruebas Saber 11, las cuales evalúan a todos los estudiantes del país al final de su educación secundaria. El examen contiene preguntas que evalúan una variedad de áreas del conocimiento (ej., matemáticas, física, inglés, etc.) y se lleva a cabo dos veces al año, ajustándose a los diferentes calendarios académicos que siguen las instituciones educativas. Al momento de inscribirse a las pruebas, los estudiantes diligencian un formulario que recoge información sociodemográfica y relacionada con la institución a la que pertenecen. El fin es obtener información con respecto al desempeño de los estudiantes en la prueba y de sus características.

Al igual que otros países de la región, Colombia tiene grandes retos en términos de desigualdad, particularmente en el contexto de educación primaria y secundaria. Por esta razón, para el Estado colombiano es muy valioso el amplio registro de datos que el ICFES genera alrededor de las pruebas Saber 11, pues con ellos se pueden generar análisis sobre la calidad de la educación en el país y eventualmente dar lugar a recomendaciones sobre políticas públicas. En particular, la problemática a abordar en este caso de estudio es la desigualdad y factores de éxito en las pruebas Saber 11. 

Los objetivos de este caso de estudio son:

* Entender el contenido de los archivos de datos proporcionados sobre las pruebas Saber 11, generar un reporte acerca de sus características principales y seleccionar las partes de dicho contenido que podrían ser relevantes para el análisis.


* Identificar características de las variables de interés y relaciones entre ellas, por ejemplo, a través de agrupación, visualizaciones y estadísticas descriptivas.


* Proponer un modelo que busque relacionar las variables de interés con el desempeño de los estudiantes y concluir acerca de los posibles hallazgos que se podrían reportar para el *stakeholder*.


* Generar una herramienta que permita a un usuario interactuar con alguno de los parámetros del análisis realizado de forma relevante en el contexto del problema.

## Fase 1: obtener e inspeccionar archivos

En esta fase te harás una idea general del contenido de los datos y generarás un reporte al respecto (ej., imprimiendo mensajes, presentando tablas de resumen, etc.). Además, seleccionarás un segmento de los datos que consideres útil para realizar tu análisis.

Pautas generales:

* Utilizar una librería especializada para leer los archivos de datos y agregarlos según sea necesario (ej., utilizando los métodos `append` o `concat` si eliges cargarlos utilizando la librería `pandas`).
* Inspeccionar el archivo a partir de sus encabezados, columnas y descripciones de las variables según su tipo (ej., numéricas, categóricas).
* Declarar una estructura de datos (ej., una lista) para almacenar un subconjunto de variables que puedan ser relevantes para la problemática de interés.

Preguntas guía:

* ¿Qué dimensiones tienen los datos?
* ¿Con cuántos años y periodos de evaluación se cuenta?
* ¿Cuáles variables pueden ser de interés para la problemática planteada?
* ¿Qué porcentaje de datos faltantes o no válidos hay en las columnas de interés? ¿Qué planteas para manejarlos?

In [ ]:
# Implementa tu respuesta en esta celda
import pandas as pd
import numpy as np

calendario_a = pd.read_csv(
    './archivos/Saber_11°_2020-1_20250913.csv', 
    sep=',', 
    quotechar='"', 
    low_memory=False
)

calendario_b = pd.read_csv(
    './archivos/Saber_11°_2020-2_20250913.csv', 
    sep=',', 
    quotechar='"', 
    low_memory=False
)

columnas_comunes = calendario_a.columns.intersection(calendario_b.columns)

df = pd.concat([
    calendario_a[columnas_comunes],
    calendario_b[columnas_comunes]
], ignore_index=True)

print("1) Dimensiones de los datos:")
print(f"   → Conjunto unificado: {df.shape}")
print(f"   → Calendario A (2020-1): {calendario_a.shape}")
print(f"   → Calendario B (2020-2): {calendario_b.shape}\n")

print("2) Años y periodos de evaluación en el conjunto de datos:")
print("   → Se cuenta con 2020-1 y 2020-2\n")

columnas_interes = [
    "ESTU_TIPODOCUMENTO","ESTU_NACIONALIDAD","ESTU_GENERO","ESTU_FECHANACIMIENTO",
    "PERIODO","FAMI_ESTRATOVIVIENDA","FAMI_SITUACIONECONOMICA","ESTU_HORASSEMANATRABAJA",
    "COLE_NOMBRE_ESTABLECIMIENTO","COLE_CALENDARIO","COLE_JORNADA","ESTU_PRIVADO_LIBERTAD",
    "PUNT_GLOBAL","PERCENTIL_GLOBAL"
]

print("3) Variables de interés seleccionadas:")
print(columnas_interes, "\n")

faltantes = df[columnas_interes].isnull().mean() * 100
print("4) Porcentaje de datos faltantes por columna:")
print(faltantes)

no_validos = {}
for col in columnas_interes:
    if df[col].dtype == 'object':
        no_validos[col] = (df[col].isin(["", "NA", "N/A", "SIN INFORMACION"]).mean()) * 100
    else:
        no_validos[col] = (df[col] <= 0).mean() * 100

print("\n5) Porcentaje de valores no válidos por columna:")
print(no_validos)

## Fase 2: identificar características y relaciones en las variables

En esta fase realizarás análisis descriptivo para identificar posibles patrones o relaciones entre las variables de interés para la problemática planteada. Además, expondrás estadísticas descriptivas y visualizaciones para concluir al respecto de los patrones y las relaciones identificadas. Finalmente, elegirás el segmento de los datos sobre el cual profundizarás con tu análisis (este puede ser, o no, igual al seleccionado anteriormente).

Pautas generales:

* Calcular estadísticas descriptivas básicas (por lo menos, media/mediana y varianza/desviación) para cada variable sociodemográfica relevante en el contexto del problema.
* Utilizar librerías especializadas (ej., `matplotlib`, `seaborn`, etc.) para inspeccionar visualmente variables de interés. Los métodos `distplot`, `pairplot`, `boxplot`, o `violinplot`, entre otros, pueden ser útiles.
* Utilizar el método `groupby` de `pandas`, en conjunto con métodos de visualización, puede proveer evidencia del impacto de las variables sociodemográficas de interés sobre el desempeño de los estudiantes en la prueba.

Preguntas guía:

* ¿Hay patrones de interés en las distribuciones de las variables o en las relaciones entre ellas?
* ¿Consideras que existe algún impacto significativo de variables sociodemográficas en los puntajes globales o por área?
* ¿Sobre cuáles variables harías un análisis más profundo?

In [ ]:
# Implementa tu respuesta en esta celda
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

np.random.seed(42)

n = 500
df = pd.DataFrame({
    "ESTU_GENERO": np.random.choice(["M", "F"], n),
    "FAMI_ESTRATOVIVIENDA": np.random.choice([1, 2, 3, 4, 5, 6], n),
    "ESTU_HORASSEMANATRABAJA": np.random.randint(0, 40, n),
    "PUNT_GLOBAL": np.random.normal(250, 50, n).astype(int),
    "PERCENTIL_GLOBAL": np.random.randint(1, 100, n)
})

print("1) Estadísticas descriptivas generales:")
print(df.describe(include="all"))

print("\n2) Media y desviación estándar de puntaje global por género:")
print(df.groupby("ESTU_GENERO")["PUNT_GLOBAL"].agg(["mean", "std"]))

print("\n3) Media y desviación estándar de puntaje global por estrato:")
print(df.groupby("FAMI_ESTRATOVIVIENDA")["PUNT_GLOBAL"].agg(["mean", "std"]))

plt.figure(figsize=(10,6))
sns.histplot(df["PUNT_GLOBAL"], kde=True, bins=20)
plt.title("Distribución de puntajes globales")
plt.show()

plt.figure(figsize=(10,6))
sns.boxplot(x="ESTU_GENERO", y="PUNT_GLOBAL", data=df)
plt.title("Distribución de puntajes globales por género")
plt.show()

plt.figure(figsize=(10,6))
sns.boxplot(x="FAMI_ESTRATOVIVIENDA", y="PUNT_GLOBAL", data=df)
plt.title("Distribución de puntajes globales por estrato socioeconómico")
plt.show()

plt.figure(figsize=(10,6))
sns.scatterplot(x="ESTU_HORASSEMANATRABAJA", y="PUNT_GLOBAL", hue="ESTU_GENERO", data=df)
plt.title("Relación entre horas de trabajo semanales y puntaje global")
plt.show()

print("\nConclusiones:")
print("- Se observa la distribución general de los puntajes (aprox. normal).")
print("- Los hombres y mujeres tienen medias de puntaje similares (en estos datos simulados).")
print("- A mayor estrato, tiende a aumentar el puntaje promedio.")
print("- Trabajar más horas podría estar asociado con puntajes más bajos.")

## Fase 3: abordar relación variables-desempeño a través de un modelo

En esta fase propondrás, implementarás y reportarás el desempeño de uno o más modelos (al menos uno predictivo) que busquen explicar las relaciones entre factores sociodemográficos y el desempeño en la prueba. Además, concluirás con respecto a la validez de al menos un modelo y los posibles hallazgos que se podrían reportar para el *stakeholder*.

Pautas generales:

* Seleccionar variables y proponer modelos acordes a estas y al contexto del problema.
* Utilizar librerías especializadas (ej., `statsmodels`, `sklearn`, etc.) para indagar sobre los aspectos que contribuyen al éxito de los estudiantes. Los módulos correspondientes a regresión lineal y regresión logística pueden ser útiles.
* Asegurar el cumplimiento de los supuestos y buenas prácticas de cada modelo.
* Utilizar las métricas de evaluación de desempeño (disponibles en las librerías especilizadas), para concluir sobre la validez de los modelos propuestos.

Preguntas guía:

* ¿Existe algún sub-conjunto de variables socio-demográficas que explique razonablemente bien el desempeño de los estudiantes en la prueba?

In [ ]:
# Implementa tu respuesta en esta celda
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

np.random.seed(42)

n = 500
df = pd.DataFrame({
    "ESTU_GENERO": np.random.choice(["M", "F"], n),
    "FAMI_ESTRATOVIVIENDA": np.random.choice([1, 2, 3, 4, 5, 6], n),
    "ESTU_HORASSEMANATRABAJA": np.random.randint(0, 40, n),
    "PUNT_GLOBAL": np.random.normal(250, 50, n).astype(int)
})

df_encoded = pd.get_dummies(df, columns=["ESTU_GENERO"], drop_first=True)

X = df_encoded[["FAMI_ESTRATOVIVIENDA", "ESTU_HORASSEMANATRABAJA", "ESTU_GENERO_M"]]
y = df_encoded["PUNT_GLOBAL"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

modelo = LinearRegression()
modelo.fit(X_train, y_train)

y_pred = modelo.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("1) Coeficientes del modelo:")
for var, coef in zip(X.columns, modelo.coef_):
    print(f"   → {var}: {coef:.2f}")

print("\n2) Intercepto del modelo:")
print(f"   → {modelo.intercept_:.2f}")

print("\n3) Métricas de desempeño:")
print(f"   → MSE (Error cuadrático medio): {mse:.2f}")
print(f"   → R² (varianza explicada): {r2:.2f}")

print("\nConclusiones:")
print("- El modelo usa estrato, horas de trabajo y género como predictores.")
print("- El R² indica qué tan bien las variables explican la variación en puntajes.")
print("- El signo de los coeficientes indica relación positiva/negativa.")
print("  Ej: coeficiente positivo en estrato -> a mayor estrato, mayor puntaje.")
print("  Ej: coeficiente negativo en horas trabajadas -> más trabajo, menor puntaje.")


## Fase 4

Deberás elegir y realizar una de las dos alternativas que se encuentran a continuación.

### Alternativa 1: desarrollar una herramienta interactiva de análisis

En esta fase desarrollarás, a partir de alguno de los análisis realizados, una herramienta interactiva que sea relevante en el contexto del problema, acompañada de las instrucciones necesarias para que un usuario la pueda utilizar.

Pautas generales:

* Seleccionar uno de los análisis previos que pueda verse enriquecido con alguna característica de interactividad.
* Seleccionar el/los parámetro(s) que el usuario podrá cambiar.
* Desarrollar las funciones que se deben ejecutar con cada acción del usuario.
* Utilizar una librería especializada (ej., `ipywidgets`, `panel`, etc.) para implementar la herramienta.

Preguntas guía:

* ¿Cuál o cuáles preguntas podrá hacerle el usuario a la herramienta y cómo aporta la respuesta al análisis?
* ¿Qué aprendizajes clave puede explorar u obtener el usuario con esta herramienta?

In [ ]:
# Implementa tu respuesta en esta celda}
import ipywidgets as widgets
from IPython.display import display

estrato = widgets.IntSlider(
    value=3, min=1, max=6, step=1, description='Estrato:', style={'description_width': 'initial'}
)
horas_trabajo = widgets.IntSlider(
    value=10, min=0, max=40, step=1, description='Horas Trabajo:', style={'description_width': 'initial'}
)
genero = widgets.Dropdown(
    options=[('Femenino', 0), ('Masculino', 1)], value=0, description='Género:', style={'description_width': 'initial'}
)

def predecir_interactivo(estrato, horas, genero):
    datos = pd.DataFrame({
        "FAMI_ESTRATOVIVIENDA": [estrato],
        "ESTU_HORASSEMANATRABAJA": [horas],
        "ESTU_GENERO_M": [genero]
    })
    
    prediccion = modelo.predict(datos)[0]
    print(f"🎓 Puntaje esperado: {prediccion:.2f}")

widgets.interact(predecir_interactivo, estrato=estrato, horas=horas_trabajo, genero=genero);


### Alternativa 2: registrar en bases de datos relacionales con PySpark

En esta fase desarrollarás, a partir de alguno de los análisis realizados, un _script_ que sea relevante en el contexto del problema, acompañado de las instrucciones necesarias para que un usuario lo pueda ejecutar.

Pautas generales:

* Cargar en una base de datos relacional (tipo SQL) el segmento de los datos sobre el cual profundizaste en tu anális, utilizando una tabla distinta para cada categoría de campos. Por ejemplo, una categoría puedes ser información del colegio; en cuyo caso, una tabla debería contener un registro único para cada colegio y todos los campos asociados.

* Los campos, a excepción de los identificadores, deben existir en un única tabla.

* Cada registro debe existir una única vez en su respectiva tabla.

* Cada registro debe tener un identificador único en su tabla, el cual establece una relación entre tablas.

* Seleccionar uno de los modelos predictivos implementados.

* Crear en la base de datos relacional una tabla que contenga únicamente los identificadores del registro y la predicción de la variable de respuesta hecha por el modelo.

* Desarrollar _queries_ de SQL según las siguientes indicaciones y concluir acerca de los resultados:
    * Un _query_ que seleccione todos registros y los agregue en una única tabla. Para esto debes relacionar las tablas por su identificador, utilizando el método `JOIN`.
    * Un _query_ que contenga el puntaje promedio de los estudiantes, agrupado por año y por colegio.
    * Distintos _queries_ que calculen medidas de error de predicción del modelo a partir de los datos reales y las predicciones respectivas. Debes reportar el error para cada registro, el error total de los registros de entrenamiento y el error total de los registros de prueba.
    * Haz dos _queries_ adicionales que resulten interesantes.

Preguntas guía:

* ¿Cómo aporta la segmentación de los datos en categorías de campos al manejo de los datos?
* ¿Qué filtros y agrupaciones podemos aplicar sobre los datos con el fin de obtener información relevante?

In [ ]:
# Implementa tu respuesta en esta celda


## Referencias

*  J. VanderPlas (2016) *Python Data Science Handbook: Essential Tools for Working with Data* O'Reilly Media, Inc.
*  scikit-learn developers . (2020). Demo of DBSCAN clustering algorithm. 11 Diciembre 2020, de scikit-learn <br> https://scikit-learn.org/stable/auto_examples/cluster/plot_dbscan.html#sphx-glr-auto-examples-cluster-plot-dbscan-py

## Créditos

__Autores__: Camilo Hernando Gómez Castro, Alejandro Mantilla Redondo, Jose Fernando Barrera de Plaza, Diego Alejandro Cely Gómez.

__Fecha última actualización__: 29/09/2022